<a href="https://colab.research.google.com/github/hna12/Topic_Modeling/blob/main/topic_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-nanum is already the newest version (20170925-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 31 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/var/cache/fontconfig: cleaning cache directory
/root/.cache/fontconfig: not cleaning non-existent 

In [7]:
# 예제 DATA
!gdown https://drive.google.com/uc?id=1KuBG40WNpVPV1ilfGaiCI2D3l8JVSfbS

Downloading...
From: https://drive.google.com/uc?id=1KuBG40WNpVPV1ilfGaiCI2D3l8JVSfbS
To: /content/train_original.json
100% 800M/800M [00:08<00:00, 89.4MB/s]


In [ ]:
import pandas as pd
import re
from tqdm import tqdm

In [8]:
df_news = pd.read_json("./train_original.json")
df_news

,media,id,article_original,abstractive,extractive
0,부산일보,360972161,[지난해 고령화와 유례가 드문 겨울 한파 등 영향으로 우리나라 사망자 수가 통계 작...,통계청이 발표한 '2018년 사망원인통계'를 보면 지난해 총 사망자 수는 관련 통계...,"[4, 11, 18]"
1,중도일보,356659913,[서산시의회(의장 임재관) 가충순·이수의 의원이 (사)한국지역신문협회에서 수여하는 ...,서산시 가충순 의원과 이수의 의원이 활발한 의정활동을 펼친 감사의 표시로 한국지역신...,"[1, 3, 4]"
2,무등일보,351718460,[지난 2004년 시작해 조선대 학생들의 대표적인 행사로 자리매김한 ‘조선대 국토대...,‘조선대의 새로운 비상을 꿈꾸다’를 슬로건으로 진행되어 단체생활을 통해 협동심과 ...,"[0, 2, 4]"
3,이데일리,335868123,[서울시는 신학기가 시작되는 다음달 4일부터 고등학교 3학년 무상급식을 실시한다고 ...,서울시가 다음달 4일부터 서울 시내 319개 고등학교 3학년 8만4700명을 대상으...,"[0, 1, 2]"
4,서울신문,351443347,[미국인 선교사가 우간다에서 의사 행세를 하며 의료 시설을 운영한 혐의로 지역 시민...,미국인 선교사가 우간다에서 의사 행세를 하며 두 아이의 죽음과 관련돼 있다며 지역 ...,"[0, 1, 2]"
...,...,...,...,...,...
260692,매일경제,353316985,"[두통과 어지럼증은 우리에게 매우 익숙하다., 하루 이틀 정도만 참으면 자연스럽게 ...","두통과 어지럼증은 우리에게 매우 익숙한 증상이며, 이 두통은 크게 긴장성 두통, 혈...","[3, 12, 16]"
260693,부산일보,360784906,[파주 최대 테마파크 퍼스트가든이 ㈜투바앤의 캐릭터 '라바'와 할로윈 페스티벌(사진...,파주 최대 테마파크 퍼스트가든이 오는 10월 12일부터 31일까지 가족들이 함께 즐...,"[0, 4, 1]"
260694,부산일보,357253753,[민원과 난공사로 완공이 늦어지고 있는 양산시 한일아파트 인근 옹벽 보수·보강공사 ...,양산시는 지난해 10월부터 43억 원을 들여 진행 중인 한일아파트 뒤편 옹벽 보수·...,"[4, 6, 7]"
260695,매일신문,347655953,"[사단법인 전국자원봉사연맹이 호국보훈의 달을 맞이하여, 오는 2019년 6월 23일...","국가를 위해 헌신한 분들의 숭고한 희생을 기리고, 유가족 들을 위로하기 위해 열리는...","[0, 1, 2]"


## 1.Text preprocessing

한글, 영어, 숫자(소수와 퍼센트) 만 남기고 기타 특수문자들을 제거

In [9]:
def text_cleaning(doc):
  text = re.sub('[^가-힣ㄱ-ㅣa-zA-Z|0-9.%]', ' ', string = doc)
  return text

In [10]:
# 원문
doc = ' '.join(df_news['article_original'].iloc[0]) #하나의 문자열로 만드는 과정
doc

'지난해 고령화와 유례가 드문 겨울 한파 등 영향으로 우리나라 사망자 수가 통계 작성 이후 가장 많았다. 폐렴과 치매의 일종인 알츠하이머병은 지난해 사망원인 순위 3위와 9위로 전년보다 각각 한 단계, 두 단계 상승하는 등 노인성 질병에 의한 사망률이 급증하는 추세다. ‘연령표준화 사망률’(표준인구 10만 명당 사망자 수)은 울산·충북·부산 순으로 높게 나타났다. ■작년 사망자 29만 8820명, 역대 최다 24일 통계청이 발표한 \'2018년 사망원인통계\'를 보면 지난해 총 사망자 수는 전년 대비 4.7%(1만 3286명) 증가한 29만 8820명으로 관련 통계를 작성한 1983년 이후 가장 많았으며, 5년 연속 증가세를 보였다. 통계청은 인구 구조의 고령화와 지난해 1~2월 유례가 드문 한파 등을 그 원인으로 꼽았다. 지난해 조사망률(인구 10만 명당 사망자 수) 역시 582.5명으로 전년보다 4.5%(25.1명) 증가해 5년 연속 늘었다. 특히 80세 이상의 사망자가 전체 사망자의 절반에 가까운 46.3%로, 10년 전보다 14.3%포인트(P)나 증가했다. ■폐렴·알츠하이머병 사망률 순위 ‘껑충’ 사망원인별로 보면 지난해 암(악성신생물)에 의한 사망률(이하 인구 10만 명당 사망자 수)은 154.3명으로 전년보다 0.2% 증가했다. 1983년 관련 통계를 집계한 이래 줄곧 암이 사망원인 1위로 집계됐다. 특히 폐렴(4위→3위)과 치매의 일종인 알츠하이머병(11위→9위)에 의한 사망률 순위 상승이 두드러졌다. 폐렴 사망률은 2004년 10위에서 꾸준히 순위가 상승하고 있고, 알츠하이머병 사망률 역시 통계 작성 이래 10대 사인에 처음 포함됐다. 지난해 알츠하이머병에 의한 사망률은 12.0명으로 전년(9.8명) 대비 22.5% 증가했다. 알츠하이머병 사망률은 10년 전(3.8명)과 비교하면 무려 214.2% 증가했다. 폐렴 사망률은 45.4명으로 전년(37.8명) 대비 20.0% 증가했다. 알코올 관련 사망률은 9.6명으로 전년보다 2.0% 늘었다. ■자살

In [11]:
# 한글,영어, 숫자만 남은 문서입니다. 
clean_doc = text_cleaning(doc)
clean_doc

'지난해 고령화와 유례가 드문 겨울 한파 등 영향으로 우리나라 사망자 수가 통계 작성 이후 가장 많았다. 폐렴과 치매의 일종인 알츠하이머병은 지난해 사망원인 순위 3위와 9위로 전년보다 각각 한 단계  두 단계 상승하는 등 노인성 질병에 의한 사망률이 급증하는 추세다.  연령표준화 사망률  표준인구 10만 명당 사망자 수 은 울산 충북 부산 순으로 높게 나타났다.  작년 사망자 29만 8820명  역대 최다 24일 통계청이 발표한  2018년 사망원인통계 를 보면 지난해 총 사망자 수는 전년 대비 4.7% 1만 3286명  증가한 29만 8820명으로 관련 통계를 작성한 1983년 이후 가장 많았으며  5년 연속 증가세를 보였다. 통계청은 인구 구조의 고령화와 지난해 1 2월 유례가 드문 한파 등을 그 원인으로 꼽았다. 지난해 조사망률 인구 10만 명당 사망자 수  역시 582.5명으로 전년보다 4.5% 25.1명  증가해 5년 연속 늘었다. 특히 80세 이상의 사망자가 전체 사망자의 절반에 가까운 46.3%로  10년 전보다 14.3%포인트 P 나 증가했다.  폐렴 알츠하이머병 사망률 순위  껑충  사망원인별로 보면 지난해 암 악성신생물 에 의한 사망률 이하 인구 10만 명당 사망자 수 은 154.3명으로 전년보다 0.2% 증가했다. 1983년 관련 통계를 집계한 이래 줄곧 암이 사망원인 1위로 집계됐다. 특히 폐렴 4위 3위 과 치매의 일종인 알츠하이머병 11위 9위 에 의한 사망률 순위 상승이 두드러졌다. 폐렴 사망률은 2004년 10위에서 꾸준히 순위가 상승하고 있고  알츠하이머병 사망률 역시 통계 작성 이래 10대 사인에 처음 포함됐다. 지난해 알츠하이머병에 의한 사망률은 12.0명으로 전년 9.8명  대비 22.5% 증가했다. 알츠하이머병 사망률은 10년 전 3.8명 과 비교하면 무려 214.2% 증가했다. 폐렴 사망률은 45.4명으로 전년 37.8명  대비 20.0% 증가했다. 알코올 관련 사망률은 9.6명으로 전년보다 2.0% 늘었다.  자살률 

형태소 분석기'Mecab' 사용

- 모든 NLP 의 필수 처리 과정
- input으로 입력되는 문장을 작은 단위로 쪼개줌
- 가장 많이 사용되는 형태소 분석기: mecab

In [ ]:
# 코랩에서 Mecab 설치 
!apt-get update
!apt-get install g++ openjdk-8-jdk 
!pip3 install konlpy JPype1-py3
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

In [14]:
# Mecab 토큰화 
from konlpy.tag import Mecab

tokenizer = Mecab()
tokenizer.morphs(clean_doc.split('.')[0]) # 첫 문장 토큰화 

['지난해',
 '고령화',
 '와',
 '유례',
 '가',
 '드문',
 '겨울',
 '한파',
 '등',
 '영향',
 '으로',
 '우리나라',
 '사망자',
 '수',
 '가',
 '통계',
 '작성',
 '이후',
 '가장',
 '많',
 '았',
 '다']

In [15]:
# 토큰화 이후 단어 길이가 2 이상인 명사만 남기기 
# 일반명사(NNG), 고유명사(NNP), 외국어(SL)만을 반환
# 한국어의 경우 토큰화 과정에서 조사나 접속사 제거로 대부분의 불용어가제거 되지만 불필요한 명사나 형용사를 제거하고 싶은 경우 불용어 제거를 추가하면 됩니다. 

from konlpy.tag import Mecab

def text_preprocess(doc):
  tokenizer = Mecab()
  
  # 텍스트 전처리
  clean_doc =  text_cleaning(doc)
  
  # 품사 붙이기
  token_tagged = tokenizer.pos(clean_doc) #[(단어, 품사)] -> [(알디노, 명사),(알파코, 명사)] 

  # 명사추출
  nouns = [s for s, t in token_tagged if t in ['SL', 'NNG', 'NNP'] and len(s) > 1] #sl: 외래어, nng:일반명사, nnp:고유명사. 보통 명사가 아닌데 명사로 태깅하는게 많아서 두개이상인것만 뽑음

  # 한 문장으로 합치기 
  # 아래에서 활용할 TfidfVectorizer는 기본적으로 토큰화가 되어있지 않은 텍스트 데이터를 입력으로 사용하기 때문에 join 작업을 미리 해줍니다. 
  preprocessed = ' '.join(nouns) #즉, 하나의 문자열로 합쳐줌

  return preprocessed

print('전처리 결과 확인 :')
text_preprocess(doc)

전처리 결과 확인 :


'지난해 고령화 유례 겨울 한파 영향 우리나라 사망자 통계 작성 이후 폐렴 치매 일종 알츠하이머병 지난해 사망 원인 순위 전년 단계 단계 상승 노인 질병 사망 급증 추세 연령 표준 사망률 표준 인구 사망자 울산 충북 부산 작년 사망자 역대 최다 통계청 발표 망원 통계 지난해 사망자 전년 대비 증가 관련 통계 작성 이후 연속 증가세 통계청 인구 구조 고령화 지난해 유례 한파 원인 지난해 조사망 인구 사망자 전년 증가 연속 이상 사망자 전체 사망자 절반 포인트 증가 폐렴 알츠하이머병 사망률 순위 사망 원인 지난해 악성 생물 사망률 이하 인구 사망자 전년 증가 관련 통계 집계 사망 원인 집계 폐렴 치매 일종 알츠하이머병 사망률 순위 상승 폐렴 사망 순위 상승 알츠하이머병 사망 통계 작성 사인 처음 포함 지난해 알츠하이머병 사망률 전년 대비 증가 알츠하이머병 사망 비교 증가 폐렴 사망 전년 대비 증가 알코올 관련 사망 전년 자살 증가 베르테르 효과 영향 지난해 자살 사망자 전년 증가 자살 전년 증가 자살 연속 증가세 자살 사망 원인 순위 차지 기록 김진 통계청 인구 동향 과장 자살 베르테르 효과 유명 자살 영향 이후 유명 자살 자살 지난해 유명 자살 영향 설명 지역 연령 구조 차이 표준 사망률 표준 인구 사망자 울산 충북 부산 서울 경기 사인 연령 표준 사망 지역 경남 심장 질환 경남 뇌혈관 질환 울산 폐렴 경북 운수 사고 전남 고의 자해 자살 충남'

In [16]:
# tqdm.pandas를 활용하면 pandas내 apply 작업의 진행상황을 확인할 수 있습니다. 
tqdm.pandas() 

df_news['article_preprocessed']= df_news['article_original'].progress_apply(lambda x: text_preprocess(' '.join(x))) #입력받은애 문자열로 합치기
df_news

100%|██████████| 260697/260697 [37:41<00:00, 115.28it/s]


,media,id,article_original,abstractive,extractive,article_preprocessed
0,부산일보,360972161,[지난해 고령화와 유례가 드문 겨울 한파 등 영향으로 우리나라 사망자 수가 통계 작...,통계청이 발표한 '2018년 사망원인통계'를 보면 지난해 총 사망자 수는 관련 통계...,"[4, 11, 18]",지난해 고령화 유례 겨울 한파 영향 우리나라 사망자 통계 작성 이후 폐렴 치매 일종...
1,중도일보,356659913,[서산시의회(의장 임재관) 가충순·이수의 의원이 (사)한국지역신문협회에서 수여하는 ...,서산시 가충순 의원과 이수의 의원이 활발한 의정활동을 펼친 감사의 표시로 한국지역신...,"[1, 3, 4]",서산시 의회 의장 임재 가충 이수 의원 한국 지역 신문협회 수여 우수 대상 가충 의...
2,무등일보,351718460,[지난 2004년 시작해 조선대 학생들의 대표적인 행사로 자리매김한 ‘조선대 국토대...,‘조선대의 새로운 비상을 꿈꾸다’를 슬로건으로 진행되어 단체생활을 통해 협동심과 ...,"[0, 2, 4]",시작 조선대 학생 대표 행사 자리매김 조선대 국토 대장정 해단식 일정 마감 조선대 ...
3,이데일리,335868123,[서울시는 신학기가 시작되는 다음달 4일부터 고등학교 3학년 무상급식을 실시한다고 ...,서울시가 다음달 4일부터 서울 시내 319개 고등학교 3학년 8만4700명을 대상으...,"[0, 1, 2]",서울시 신학기 시작 다음 고등학교 학년 무상급식 실시 서울 시내 고등학교 학년 대상...
4,서울신문,351443347,[미국인 선교사가 우간다에서 의사 행세를 하며 의료 시설을 운영한 혐의로 지역 시민...,미국인 선교사가 우간다에서 의사 행세를 하며 두 아이의 죽음과 관련돼 있다며 지역 ...,"[0, 1, 2]",미국인 선교사 우간다 의사 행세 의료 시설 운영 혐의 지역 시민 단체 주민 고소 원...
...,...,...,...,...,...,...
260692,매일경제,353316985,"[두통과 어지럼증은 우리에게 매우 익숙하다., 하루 이틀 정도만 참으면 자연스럽게 ...","두통과 어지럼증은 우리에게 매우 익숙한 증상이며, 이 두통은 크게 긴장성 두통, 혈...","[3, 12, 16]",두통 어지럼증 하루 이틀 정도 자연 증상 경우 신체 기관 중요 질환 전조 증상 간과...
260693,부산일보,360784906,[파주 최대 테마파크 퍼스트가든이 ㈜투바앤의 캐릭터 '라바'와 할로윈 페스티벌(사진...,파주 최대 테마파크 퍼스트가든이 오는 10월 12일부터 31일까지 가족들이 함께 즐...,"[0, 4, 1]",파주 최대 테마파크 퍼스트 가든 투바 캐릭터 라바 할로윈 페스티벌 사진 진행 이번 ...
260694,부산일보,357253753,[민원과 난공사로 완공이 늦어지고 있는 양산시 한일아파트 인근 옹벽 보수·보강공사 ...,양산시는 지난해 10월부터 43억 원을 들여 진행 중인 한일아파트 뒤편 옹벽 보수·...,"[4, 6, 7]",민원 난공사 완공 양산 아파트 인근 옹벽 보수 보강 공사 현장 경남 양산시 평산동 ...
260695,매일신문,347655953,"[사단법인 전국자원봉사연맹이 호국보훈의 달을 맞이하여, 오는 2019년 6월 23일...","국가를 위해 헌신한 분들의 숭고한 희생을 기리고, 유가족 들을 위로하기 위해 열리는...","[0, 1, 2]",사단 법인 전국 자원 봉사 연맹 호국 보훈 일요일 대구 두류 공원 야외 음악당 진행...


In [17]:
# 전처리 완료된 데이터프레임을 피클파일로 저장
df_news.to_pickle('./news_tokenized.pickle')
df_news

,media,id,article_original,abstractive,extractive,article_preprocessed
0,부산일보,360972161,[지난해 고령화와 유례가 드문 겨울 한파 등 영향으로 우리나라 사망자 수가 통계 작...,통계청이 발표한 '2018년 사망원인통계'를 보면 지난해 총 사망자 수는 관련 통계...,"[4, 11, 18]",지난해 고령화 유례 겨울 한파 영향 우리나라 사망자 통계 작성 이후 폐렴 치매 일종...
1,중도일보,356659913,[서산시의회(의장 임재관) 가충순·이수의 의원이 (사)한국지역신문협회에서 수여하는 ...,서산시 가충순 의원과 이수의 의원이 활발한 의정활동을 펼친 감사의 표시로 한국지역신...,"[1, 3, 4]",서산시 의회 의장 임재 가충 이수 의원 한국 지역 신문협회 수여 우수 대상 가충 의...
2,무등일보,351718460,[지난 2004년 시작해 조선대 학생들의 대표적인 행사로 자리매김한 ‘조선대 국토대...,‘조선대의 새로운 비상을 꿈꾸다’를 슬로건으로 진행되어 단체생활을 통해 협동심과 ...,"[0, 2, 4]",시작 조선대 학생 대표 행사 자리매김 조선대 국토 대장정 해단식 일정 마감 조선대 ...
3,이데일리,335868123,[서울시는 신학기가 시작되는 다음달 4일부터 고등학교 3학년 무상급식을 실시한다고 ...,서울시가 다음달 4일부터 서울 시내 319개 고등학교 3학년 8만4700명을 대상으...,"[0, 1, 2]",서울시 신학기 시작 다음 고등학교 학년 무상급식 실시 서울 시내 고등학교 학년 대상...
4,서울신문,351443347,[미국인 선교사가 우간다에서 의사 행세를 하며 의료 시설을 운영한 혐의로 지역 시민...,미국인 선교사가 우간다에서 의사 행세를 하며 두 아이의 죽음과 관련돼 있다며 지역 ...,"[0, 1, 2]",미국인 선교사 우간다 의사 행세 의료 시설 운영 혐의 지역 시민 단체 주민 고소 원...
...,...,...,...,...,...,...
260692,매일경제,353316985,"[두통과 어지럼증은 우리에게 매우 익숙하다., 하루 이틀 정도만 참으면 자연스럽게 ...","두통과 어지럼증은 우리에게 매우 익숙한 증상이며, 이 두통은 크게 긴장성 두통, 혈...","[3, 12, 16]",두통 어지럼증 하루 이틀 정도 자연 증상 경우 신체 기관 중요 질환 전조 증상 간과...
260693,부산일보,360784906,[파주 최대 테마파크 퍼스트가든이 ㈜투바앤의 캐릭터 '라바'와 할로윈 페스티벌(사진...,파주 최대 테마파크 퍼스트가든이 오는 10월 12일부터 31일까지 가족들이 함께 즐...,"[0, 4, 1]",파주 최대 테마파크 퍼스트 가든 투바 캐릭터 라바 할로윈 페스티벌 사진 진행 이번 ...
260694,부산일보,357253753,[민원과 난공사로 완공이 늦어지고 있는 양산시 한일아파트 인근 옹벽 보수·보강공사 ...,양산시는 지난해 10월부터 43억 원을 들여 진행 중인 한일아파트 뒤편 옹벽 보수·...,"[4, 6, 7]",민원 난공사 완공 양산 아파트 인근 옹벽 보수 보강 공사 현장 경남 양산시 평산동 ...
260695,매일신문,347655953,"[사단법인 전국자원봉사연맹이 호국보훈의 달을 맞이하여, 오는 2019년 6월 23일...","국가를 위해 헌신한 분들의 숭고한 희생을 기리고, 유가족 들을 위로하기 위해 열리는...","[0, 1, 2]",사단 법인 전국 자원 봉사 연맹 호국 보훈 일요일 대구 두류 공원 야외 음악당 진행...


In [18]:
df_news = pd.read_pickle('./news_tokenized.pickle')
df_news

,media,id,article_original,abstractive,extractive,article_preprocessed
0,부산일보,360972161,[지난해 고령화와 유례가 드문 겨울 한파 등 영향으로 우리나라 사망자 수가 통계 작...,통계청이 발표한 '2018년 사망원인통계'를 보면 지난해 총 사망자 수는 관련 통계...,"[4, 11, 18]",지난해 고령화 유례 겨울 한파 영향 우리나라 사망자 통계 작성 이후 폐렴 치매 일종...
1,중도일보,356659913,[서산시의회(의장 임재관) 가충순·이수의 의원이 (사)한국지역신문협회에서 수여하는 ...,서산시 가충순 의원과 이수의 의원이 활발한 의정활동을 펼친 감사의 표시로 한국지역신...,"[1, 3, 4]",서산시 의회 의장 임재 가충 이수 의원 한국 지역 신문협회 수여 우수 대상 가충 의...
2,무등일보,351718460,[지난 2004년 시작해 조선대 학생들의 대표적인 행사로 자리매김한 ‘조선대 국토대...,‘조선대의 새로운 비상을 꿈꾸다’를 슬로건으로 진행되어 단체생활을 통해 협동심과 ...,"[0, 2, 4]",시작 조선대 학생 대표 행사 자리매김 조선대 국토 대장정 해단식 일정 마감 조선대 ...
3,이데일리,335868123,[서울시는 신학기가 시작되는 다음달 4일부터 고등학교 3학년 무상급식을 실시한다고 ...,서울시가 다음달 4일부터 서울 시내 319개 고등학교 3학년 8만4700명을 대상으...,"[0, 1, 2]",서울시 신학기 시작 다음 고등학교 학년 무상급식 실시 서울 시내 고등학교 학년 대상...
4,서울신문,351443347,[미국인 선교사가 우간다에서 의사 행세를 하며 의료 시설을 운영한 혐의로 지역 시민...,미국인 선교사가 우간다에서 의사 행세를 하며 두 아이의 죽음과 관련돼 있다며 지역 ...,"[0, 1, 2]",미국인 선교사 우간다 의사 행세 의료 시설 운영 혐의 지역 시민 단체 주민 고소 원...
...,...,...,...,...,...,...
260692,매일경제,353316985,"[두통과 어지럼증은 우리에게 매우 익숙하다., 하루 이틀 정도만 참으면 자연스럽게 ...","두통과 어지럼증은 우리에게 매우 익숙한 증상이며, 이 두통은 크게 긴장성 두통, 혈...","[3, 12, 16]",두통 어지럼증 하루 이틀 정도 자연 증상 경우 신체 기관 중요 질환 전조 증상 간과...
260693,부산일보,360784906,[파주 최대 테마파크 퍼스트가든이 ㈜투바앤의 캐릭터 '라바'와 할로윈 페스티벌(사진...,파주 최대 테마파크 퍼스트가든이 오는 10월 12일부터 31일까지 가족들이 함께 즐...,"[0, 4, 1]",파주 최대 테마파크 퍼스트 가든 투바 캐릭터 라바 할로윈 페스티벌 사진 진행 이번 ...
260694,부산일보,357253753,[민원과 난공사로 완공이 늦어지고 있는 양산시 한일아파트 인근 옹벽 보수·보강공사 ...,양산시는 지난해 10월부터 43억 원을 들여 진행 중인 한일아파트 뒤편 옹벽 보수·...,"[4, 6, 7]",민원 난공사 완공 양산 아파트 인근 옹벽 보수 보강 공사 현장 경남 양산시 평산동 ...
260695,매일신문,347655953,"[사단법인 전국자원봉사연맹이 호국보훈의 달을 맞이하여, 오는 2019년 6월 23일...","국가를 위해 헌신한 분들의 숭고한 희생을 기리고, 유가족 들을 위로하기 위해 열리는...","[0, 1, 2]",사단 법인 전국 자원 봉사 연맹 호국 보훈 일요일 대구 두류 공원 야외 음악당 진행...


In [19]:
# 한 문장으로 join 해놓은 전처리 완료된 문장들을 공백 기준으로 분리하는 함수입니다. 

def my_tokenizer(text):
  return text.split()

In [20]:
my_tokenizer(df_news['article_preprocessed'].iloc[0][:100])

['지난해',
 '고령화',
 '유례',
 '겨울',
 '한파',
 '영향',
 '우리나라',
 '사망자',
 '통계',
 '작성',
 '이후',
 '폐렴',
 '치매',
 '일종',
 '알츠하이머병',
 '지난해',
 '사망',
 '원인',
 '순위',
 '전년',
 '단계',
 '단계',
 '상승',
 '노인',
 '질병',
 '사망',
 '급증',
 '추세',
 '연령',
 '표준']

## 2.TF-IDF 행렬 만들기
- 단어의 빈도와 역 문서 빈도를 사용해 DTM(문서 단어 행렬)내의 각 단어들마다 중요한 정도를 가중치로 주는 방법

In [21]:
# sklearn을 통해서 TF-IDF 행렬을 만들어 줍니다. 
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(tokenizer=my_tokenizer) #벡터화
# 기사 개수는 1000개로 조절
tfidf = tfidf_vect.fit_transform(raw_documents = df_news['article_preprocessed'].iloc[:1000]) 

tfidf

<1000x17985 sparse matrix of type '<class 'numpy.float64'>'
	with 99572 stored elements in Compressed Sparse Row format>

## 3.LDA 학습
- TOPIC의 개수를 20으로 해 LDA모델 학습

In [22]:
from sklearn.decomposition import LatentDirichletAllocation #sklearn에도 젠심? 거기서도 LDA 가능하다함.

lda = LatentDirichletAllocation(n_components =20, # 몇개의 단어 덩어리를 뽑을건지 
                                max_iter = 20,
                                learning_method='online', # 미니배치를 보겠다는 게 'ONLINE'
                                random_state=100) # 고정값

#전체에서 하나의 통일적인 주제를 뽑는거기때문에 accuracy가 중요하지 않음. 
#보조지표 1.혼잡도-> 낮을수록 좋음, 2.토픽응집도-> 높을수록 좋음
#토픽수가 중요. 우리가 바꿀수있는것으로 맘에 안들경우 이걸 바꾸기

lda_output = lda.fit_transform(tfidf)

In [23]:
print('lda components:\n',lda.components_)  #주제를 뽑은것
print('shape:', lda.components_.shape) 
#각 주제마다 높은 확률인 애들을 뽑기 -> 주요 키워드(높은 빈도수/ 확률인 애들)를 뽑기 
#즉, 확률값이 제일 높은애의 index값을 뽑으면 됨

lda components:
 [[0.05008091 0.05010153 0.05010842 ... 0.05010621 0.05010268 0.05009866]
 [0.05008622 0.05010681 0.05008638 ... 0.05011008 0.05009655 0.05010943]
 [0.05009212 0.05010949 0.05010603 ... 0.05010977 0.05008111 0.05011496]
 ...
 [0.05010224 0.05008602 0.05011224 ... 0.05009488 0.05010439 0.05009758]
 [0.22831503 0.09297862 0.13370314 ... 0.73783973 0.05015904 0.0501043 ]
 [0.05009867 0.05010542 0.05010889 ... 0.05009237 0.05009483 0.05008695]]
shape: (20, 17985)


LDA 과정에서 얻은 TOPIC들 프린트 (TOPIC당 7개의 단어를 default로 설정)

→ 즉, 상위 7개의 단어들을 뽑음

In [24]:
terms = tfidf_vect.get_feature_names() # 단어 집합. terms도 순서가 있을것

def get_topics(components, feature_names, n=7):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])
get_topics(lda.components_,terms)

#for i in topic.argsort()[:-n - 1:-1] : 한줄을 sorting해서 돌리겠다.큰애가 필요하니 뒤에 -1해줌. 높은 애들이 나옴
#slicing시 맨뒤를 -1로 두면 뒤에서부터 차례로 정렬됨
#[10,4,3,6][::-1]
#즉 간격이 -1씩이라 역순으로 -> 6,3,4,10이 나옴
#round(2) : 2째 자리에서 반올림

Topic 1: [('청남대', 0.74), ('수궁가', 0.48), ('유하영', 0.41), ('진도', 0.27), ('악원', 0.19), ('박초월', 0.19), ('사사', 0.19)]
Topic 2: [('무스탕', 0.61), ('패션쇼', 0.58), ('김대건', 0.38), ('순례', 0.32), ('버시', 0.3), ('신앙', 0.3), ('합덕', 0.25)]
Topic 3: [('오케스트라', 0.43), ('윈드', 0.38), ('일초', 0.2), ('연주', 0.16), ('igor', 0.09), ('prince', 0.09), ('dances', 0.09)]
Topic 4: [('비격진천뢰', 0.73), ('출토', 0.52), ('무덤', 0.44), ('원태', 0.35), ('흉노', 0.31), ('안타', 0.28), ('타자', 0.26)]
Topic 5: [('딤채', 0.76), ('금메달', 0.68), ('감영', 0.67), ('손호영', 0.66), ('포드', 0.56), ('종원', 0.55), ('다이빙', 0.52)]
Topic 6: [('교회', 1.19), ('접종', 1.12), ('대상포진', 0.96), ('새문안', 0.51), ('레소토', 0.37), ('덕정', 0.31), ('교단', 0.24)]
Topic 7: [('강미혜', 0.59), ('평택항', 0.57), ('와이어리스', 0.48), ('젠하이저', 0.34), ('항로', 0.32), ('김우진', 0.32), ('물동량', 0.31)]
Topic 8: [('와인', 0.68), ('청포', 0.48), ('은서', 0.45), ('너리', 0.32), ('육군', 0.3), ('oem', 0.16), ('이육사', 0.16)]
Topic 9: [('한자', 0.82), ('미래엔', 0.7), ('sb', 0.52), ('외투', 0.5), ('맥쿼리', 0.49), ('헤일', 0.48), ('아바스

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


## 4.LDA 시각화

In [25]:
# LDAvis 는 토픽 모델링에 자주 이용되는 Latent Dirichlet Allocation (LDA) 모델의 학습 결과를 시각적으로 표현하는 라이브러리입니다
!pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 5.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=b22e2e55f721b6a22d904e668aa2a5773a4e99a4180cf61420c18ab9830aafcc
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=a8eeabfe64b619a5286dc76ad6b515379765625a9f64361d46644d3581ae33e7
  Stored in directory: /root/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built pyLDAvis sklearn


In [26]:
# LDA 시각화를 진행해봅시다. 
import pyLDAvis
import pyLDAvis.sklearn

pyLDAvis.enable_notebook()
vis = pyLDAvis.sklearn.prepare(lda, tfidf, tfidf_vect, mds='tsne')
pyLDAvis.display(vis)


/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-on

✔️LDAvis의 두가지 정보
- HTML 의 좌측에 출력되는 topic의 2차원 embedding vector (비슷한 위치에 존재하는 topic들은 서로 비슷한 문맥을 지님)
- 오른쪽에 출력되는 각 topic의 keyword

✔️원들은 각각의 20개의 topic을 나타냄

✔️각 원과의 거리 = 각 topic들이 서로 얼마나 다른지를 보여줌 (만약 두 원이 겹치면 이 두개의 topic은 유사한 topic이라는 의미)

✔️각 원을 클릭시 우측에 해당 topic에 관한 정보 나타남

✔️위의 예시에서
- 1번 topic : 사업, 지역, 산업, 지원, 한국 등 가장 광범위한 키워드들을 포함
- 15번 topic : 1번 topic과 겹치는 부분 거의 없고 거리가 멀다. 이는 조광조, 영의정, 기묘사화 등 역사와 관련된 topic
- 3번 topic : 1번 topic에 포함된 것으로 코스피, 거래와 관련한 키워드를 갖고 있음.

✔️ 우측 상단의 λ
- [0, 1] 사이에서 조절가능한 값
- λ를 1 로 설정하면 토픽 별로 가장 자주 등장하는 단어들을 우선적으로 키워드로 선택한다는 의미
- λ 를 0 에 가깝게 설정할수록 토픽 간에 차이가 많이 나는 단어를 선택한다는 의미